In [1]:
import streamlit as st
from streamlit_chat import message
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
import tempfile
import whisper
from pytube import YouTube
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

from openai import AzureOpenAI

c:\Users\pavansannadi\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\__init__.py:48: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.0)
  import scipy


In [6]:

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

llm = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-05-01-preview",
    temperature=0)

def download_transcript():
    tmpdir = "C:/Tmp/"
    if not os.path.exists("transcription.txt"):
        youtube = YouTube("https://www.youtube.com/watch?v=GSsIv0GQTUo")
        audio = youtube.streams.filter(only_audio=True).first()

        audio_test_file = audio.download(output_path=tmpdir)

        client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
            api_version="2024-02-01",
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        )

        deployment_id = "whisper" #This will correspond to the custom name you chose for your deployment when you deployed a model."
        
        transcription = client.audio.transcriptions.create(
            file=open(audio_test_file, "rb"),            
            model=deployment_id
        )
        # print(transcription.text)

In [7]:
download_transcript()

with open("C:/Tmp/transcription.txt", "w") as file:
    file.write(transcription.text.strip())

In [8]:
template = """
You are an AI assistant that helps people find information.
Please respond with the summarization of the video transcript provided as input.
If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [9]:
loader = TextLoader("C:/Tmp/transcription.txt")
text_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [10]:
embeddings = AzureOpenAIEmbeddings()
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [12]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
)
chain.invoke("summarize the context?").content

'The video transcript explains the concept of the String Constant Pool (SCP) in Java. The SCP, also known as the string pool, is a special region within the heap memory dedicated to storing string literals. When a string literal is created, Java checks if it already exists in the SCP. If it does, the existing string is reused; otherwise, a new string is added to the pool. The transcript provides examples to illustrate this behavior, showing that string literals with the same value point to the same memory location in the SCP, resulting in reference equality. Conversely, strings created with the `new` keyword are stored in different memory locations, leading to reference inequality. The video encourages viewers to subscribe and stay tuned for further videos on related topics.'